# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in corpus.

In this notebook, we will be using **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

Once the topic modeling technique is applied, our job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, we can try changing up the number of topics, the terms in the document-term matrix, model parameters.

## Topic Modeling - Attempt #1 (All Text)

In [ ]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,abc,ability,able,abortion,abruptly,absolutely,absorbing,absurdities,abuse,abused,...,youngest,youtube,youve,yyou,zealand,zeppelin,zero,zombie,zuckerberg,éclair
ali,1,0,2,0,0,0,1,1,1,2,...,0,0,2,1,0,0,0,1,0,0
anthony,0,0,0,2,0,0,0,1,0,0,...,0,0,6,0,10,0,0,0,0,0
joe,0,0,2,0,0,0,0,1,1,0,...,1,3,6,0,0,0,0,0,0,0
john,0,0,3,0,0,1,0,1,0,0,...,0,0,3,0,0,0,0,0,0,1
mike,0,0,0,0,0,0,0,1,0,0,...,0,0,3,0,0,2,1,0,0,0
ronny,0,1,1,0,1,0,0,1,0,0,...,0,7,2,0,0,0,2,0,3,0


In [ ]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

In [ ]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,joe,john,mike,ronny
abc,1,0,0,0,0,0
ability,0,0,0,0,0,1
able,2,0,2,3,0,1
abortion,0,2,0,0,0,0
abruptly,0,0,0,0,0,1


In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.004*"says" + 0.004*"tell" + 0.004*"didnt" + 0.004*"little" + 0.004*"mean" + 0.004*"did" + 0.004*"lot" + 0.003*"good" + 0.003*"guys" + 0.003*"thing"'),
 (1,
  '0.007*"fcking" + 0.007*"fck" + 0.006*"theres" + 0.004*"look" + 0.004*"man" + 0.004*"real" + 0.004*"kids" + 0.003*"good" + 0.003*"life" + 0.003*"house"')]

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.009*"says" + 0.006*"jenny" + 0.006*"goes" + 0.006*"id" + 0.005*"mean" + 0.005*"point" + 0.004*"kind" + 0.004*"feel" + 0.004*"going" + 0.004*"want"'),
 (1,
  '0.005*"little" + 0.004*"lot" + 0.004*"mom" + 0.004*"old" + 0.004*"did" + 0.004*"tell" + 0.004*"guys" + 0.004*"joke" + 0.004*"thing" + 0.004*"hey"'),
 (2,
  '0.008*"fcking" + 0.008*"fck" + 0.006*"theres" + 0.005*"look" + 0.005*"man" + 0.004*"real" + 0.004*"kids" + 0.004*"good" + 0.004*"life" + 0.004*"house"')]

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.007*"ok" + 0.006*"lot" + 0.005*"gotta" + 0.005*"wanna" + 0.005*"husband" + 0.004*"women" + 0.004*"pregnant" + 0.004*"need" + 0.004*"hes" + 0.004*"god"'),
 (1,
  '0.005*"mom" + 0.004*"good" + 0.004*"didnt" + 0.004*"dad" + 0.004*"baby" + 0.004*"fun" + 0.004*"look" + 0.004*"little" + 0.004*"guys" + 0.004*"love"'),
 (2,
  '0.000*"theres" + 0.000*"good" + 0.000*"goes" + 0.000*"hes" + 0.000*"look" + 0.000*"want" + 0.000*"love" + 0.000*"guys" + 0.000*"money" + 0.000*"tell"'),
 (3,
  '0.008*"fcking" + 0.008*"fck" + 0.006*"says" + 0.006*"theres" + 0.005*"kind" + 0.005*"going" + 0.004*"mean" + 0.004*"house" + 0.004*"goes" + 0.004*"make"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

In [ ]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
# Read in the cleaned data
data_clean = pd.read_pickle('corpus.pkl')  # My corpus.pkl file contains cleaned dataframe same as data_clean.pkl
data_clean

,transcript
ali,ladies and gentlemen please welcome to the sta...
anthony,thank you thank you thank you san francisco th...
joe,ladies and gentlemen welcome joe rogan wha...
john,armed with boyish charm and a sharp wit the fo...
mike,wow hey thank you thanks thank you guys hey se...
ronny,ladies and gentlemen make some noise for r...


In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ladies gentlemen stage ali hi thank hello na s...
anthony,thank thank people i em i francisco city world...
joe,ladies gentlemen joe fck thanks phone fckface ...
john,charm wit snl writer john mulaney marriage bee...
mike,wow hey thanks look insane years everyone i id...
ronny,ladies gentlemen noise chieng thank wifes eggs...


In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = list(text.ENGLISH_STOP_WORDS.union(add_stop_words))

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,abc,ability,abortion,absurdities,abuse,accent,accents,access,accident,accolades,...,york,youd,youll,youtube,youve,zealand,zeppelin,zombie,zuckerberg,éclair
ali,1,0,0,1,0,0,0,0,1,0,...,0,0,3,0,1,0,0,1,0,0
anthony,0,0,2,1,0,1,0,0,0,0,...,0,0,0,0,2,10,0,0,0,0
joe,0,0,0,1,1,1,1,0,0,0,...,0,1,0,2,1,0,0,0,0,0
john,0,0,0,1,0,1,0,2,0,0,...,7,1,0,0,1,0,0,0,0,1
mike,0,0,0,1,0,0,0,0,15,0,...,5,2,0,0,0,0,2,0,0,0
ronny,0,1,0,1,0,1,0,0,0,1,...,8,0,2,6,2,0,0,0,3,0


In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"shit" + 0.009*"day" + 0.009*"lot" + 0.008*"fck" + 0.007*"man" + 0.007*"cause" + 0.006*"thing" + 0.006*"gon" + 0.006*"life" + 0.006*"joke"'),
 (1,
  '0.008*"way" + 0.007*"day" + 0.007*"cause" + 0.007*"mom" + 0.006*"guy" + 0.006*"point" + 0.006*"kind" + 0.005*"parents" + 0.005*"wife" + 0.005*"house"')]

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"day" + 0.009*"mom" + 0.008*"way" + 0.006*"shit" + 0.006*"lot" + 0.006*"baby" + 0.006*"cause" + 0.006*"wife" + 0.005*"money" + 0.005*"clinton"'),
 (1,
  '0.016*"joke" + 0.012*"day" + 0.008*"thing" + 0.007*"school" + 0.007*"grandma" + 0.007*"jokes" + 0.007*"family" + 0.006*"anthony" + 0.006*"lot" + 0.006*"baby"'),
 (2,
  '0.010*"cause" + 0.010*"fck" + 0.009*"kind" + 0.008*"house" + 0.008*"gon" + 0.007*"way" + 0.007*"man" + 0.007*"life" + 0.007*"things" + 0.006*"guy"')]

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.012*"clinton" + 0.009*"mom" + 0.009*"day" + 0.009*"way" + 0.008*"cow" + 0.007*"wife" + 0.007*"dad" + 0.007*"house" + 0.006*"cause" + 0.006*"thing"'),
 (1,
  '0.009*"mom" + 0.008*"money" + 0.007*"day" + 0.007*"way" + 0.007*"malaysia" + 0.007*"guy" + 0.006*"baby" + 0.006*"boomers" + 0.006*"fun" + 0.006*"country"'),
 (2,
  '0.010*"cause" + 0.009*"day" + 0.008*"fck" + 0.008*"thing" + 0.008*"way" + 0.008*"kind" + 0.008*"house" + 0.008*"things" + 0.007*"gon" + 0.007*"life"'),
 (3,
  '0.012*"shit" + 0.011*"lot" + 0.010*"husband" + 0.008*"women" + 0.007*"day" + 0.006*"dude" + 0.006*"god" + 0.005*"fingers" + 0.005*"gon" + 0.005*"cause"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ladies gentlemen welcome stage ali wong hi wel...
anthony,thank san francisco thank good people surprise...
joe,ladies gentlemen joe fck san francisco thanks ...
john,boyish charm sharp wit former snl writer john ...
mike,wow hey thanks hey seattle nice look crazy ins...
ronny,ladies gentlemen noise ronny chieng thank im i...


In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,abc,ability,able,abortion,abuse,academic,accent,accents,acceptable,access,...,young,younger,youngest,youtube,yyou,zealand,zeppelin,zombie,zuckerberg,éclair
ali,1,0,2,0,0,0,0,0,0,0,...,2,0,0,0,1,0,0,1,0,0
anthony,0,0,0,2,0,0,1,0,0,0,...,0,0,0,0,0,10,0,0,0,0
joe,0,0,2,0,1,0,1,1,0,0,...,4,0,1,3,0,0,0,0,0,0
john,0,0,3,0,0,0,1,0,1,2,...,2,1,0,0,0,0,0,0,0,1
mike,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,2,0,0,0
ronny,0,1,1,0,0,1,1,0,0,0,...,0,0,0,7,0,0,0,0,3,0


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.008*"joke" + 0.007*"jenny" + 0.006*"anthony" + 0.004*"grandma" + 0.004*"accident" + 0.003*"mad" + 0.003*"shark" + 0.003*"scrambler" + 0.003*"comedy" + 0.003*"argument"'),
 (1,
  '0.007*"fck" + 0.005*"wife" + 0.004*"clinton" + 0.004*"fcking" + 0.004*"chinese" + 0.003*"fun" + 0.003*"president" + 0.003*"husband" + 0.003*"fuck" + 0.003*"ass"')]

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.012*"clinton" + 0.008*"cow" + 0.007*"wife" + 0.005*"president" + 0.005*"john" + 0.005*"movie" + 0.004*"petunia" + 0.004*"ballroom" + 0.004*"finch" + 0.004*"office"'),
 (1,
  '0.007*"joke" + 0.006*"fun" + 0.006*"fuck" + 0.005*"anthony" + 0.005*"chinese" + 0.004*"husband" + 0.004*"internet" + 0.004*"grandma" + 0.004*"ok" + 0.004*"country"'),
 (2,
  '0.012*"fck" + 0.007*"jenny" + 0.007*"fcking" + 0.004*"texas" + 0.004*"accident" + 0.003*"weird" + 0.003*"scrambler" + 0.003*"dick" + 0.003*"water" + 0.003*"sort"')]

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.019*"joke" + 0.013*"anthony" + 0.009*"grandma" + 0.008*"mad" + 0.008*"shark" + 0.006*"san" + 0.006*"zealand" + 0.005*"party" + 0.005*"today" + 0.005*"fun"'),
 (1,
  '0.006*"chinese" + 0.006*"husband" + 0.006*"fuck" + 0.005*"fun" + 0.005*"ok" + 0.005*"internet" + 0.005*"doctor" + 0.005*"country" + 0.005*"malaysia" + 0.004*"eggs"'),
 (2,
  '0.012*"jenny" + 0.006*"accident" + 0.005*"scrambler" + 0.005*"argument" + 0.004*"andy" + 0.004*"club" + 0.004*"sort" + 0.004*"marriage" + 0.004*"flight" + 0.003*"texas"'),
 (3,
  '0.013*"fck" + 0.008*"clinton" + 0.007*"fcking" + 0.006*"president" + 0.005*"cow" + 0.005*"wife" + 0.005*"dick" + 0.004*"weird" + 0.004*"office" + 0.003*"john"')]

## Identify Topics in Each Document

Out of all the topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.010*"joke" + 0.008*"fun" + 0.007*"anthony" + 0.006*"fuck" + 0.005*"chinese" + 0.005*"internet" + 0.005*"grandma" + 0.005*"malaysia" + 0.005*"boomers" + 0.004*"mad"'),
 (1,
  '0.022*"fck" + 0.012*"fcking" + 0.006*"weird" + 0.005*"water" + 0.005*"dick" + 0.004*"em" + 0.004*"texas" + 0.004*"earth" + 0.004*"smart" + 0.004*"religion"'),
 (2,
  '0.010*"husband" + 0.010*"ok" + 0.007*"pregnant" + 0.006*"asian" + 0.005*"fingers" + 0.005*"doo" + 0.005*"wan" + 0.005*"toilet" + 0.004*"butt" + 0.004*"ass"'),
 (3,
  '0.008*"clinton" + 0.008*"jenny" + 0.005*"cow" + 0.005*"john" + 0.005*"wife" + 0.004*"accident" + 0.003*"president" + 0.003*"scrambler" + 0.003*"mean" + 0.003*"movie"')]

Topic 0 → Humor & Pop Culture

Topic 1 → Strong Language & Informal Speech

Topic 2 → Daily Life & Relationships

Topic 3 → Politics & Notable Figures

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
topics = [max(doc, key=lambda x: x[1])[0] for doc in corpus_transformed]

# Zip topics with the document index
topic_distribution = list(zip(topics, data_dtmna.index))

# Display the first few results
print(topic_distribution)


[(2, 'ali'), (0, 'anthony'), (1, 'joe'), (3, 'john'), (3, 'mike'), (0, 'ronny')]


Humor & Pop Culture (Topic 0): Anthony, Ronny

Strong Language & Informal Speech (Topic 1): Joe

Daily Life & Relationships (Topic 2): Ali

Politics & Notable Figures (Topic 3): John, Mike

## Further Analysis

1. Lets's try further modifying the parameters of the topic models above and see if we can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if we can get better topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.013*"jenny" + 0.007*"accident" + 0.006*"scrambler" + 0.005*"argument" + 0.005*"club" + 0.005*"andy" + 0.004*"sort" + 0.004*"marriage" + 0.004*"flight" + 0.004*"relationship"'),
 (1,
  '0.014*"clinton" + 0.009*"cow" + 0.009*"wife" + 0.006*"john" + 0.006*"president" + 0.006*"movie" + 0.005*"petunia" + 0.005*"ballroom" + 0.005*"finch" + 0.004*"dinner"'),
 (2,
  '0.016*"fck" + 0.011*"joke" + 0.009*"fcking" + 0.008*"anthony" + 0.006*"mad" + 0.006*"grandma" + 0.006*"weird" + 0.005*"san" + 0.005*"water" + 0.005*"shark"'),
 (3,
  '0.011*"husband" + 0.010*"ok" + 0.008*"pregnant" + 0.006*"asian" + 0.006*"fingers" + 0.006*"doo" + 0.005*"wan" + 0.005*"toilet" + 0.004*"ass" + 0.004*"butt"'),
 (4,
  '0.008*"fun" + 0.008*"chinese" + 0.008*"malaysia" + 0.007*"internet" + 0.007*"boomers" + 0.006*"country" + 0.006*"fuck" + 0.006*"eggs" + 0.006*"wife" + 0.006*"tax"')]

Topic 0 → Personal Relationships & Life Events

Topic 1 → Politics & Public Figures

Topic 2 → Profanity & Informal Speech

Topic 3 → Humor & Relationships

Topic 4 → Culture & Society

In [ ]:
def extract_verbs(text):
    '''Given a string of text, tokenize it and pull out only the verbs.'''
    is_verb = lambda pos: pos[:2] == 'VB'  
    tokenized = word_tokenize(text)  # Tokenize the text
    verbs = [word for (word, pos) in pos_tag(tokenized) if is_verb(pos)]  # Extract verbs
    return ' '.join(verbs)

In [ ]:
# Apply the function to the transcripts to filter only on verbs
data_verbs = pd.DataFrame(data_clean.transcript.apply(extract_verbs))
data_verbs

,transcript
ali,please hello thank coming are gon have get hav...
anthony,thank thank be were told was gon tape said do ...
joe,welcome rogan is going coming appreciate god d...
john,armed offers takes met wish die see thank was ...
mike,thank thank guys see look were is i know start...
ronny,make thank hawaii turning harvesting harvestin...


In [ ]:
# Create a new document-term matrix using only verbs, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_verbs.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_verbs.index
data_dtmna

,absorbing,abuse,abused,accepted,according,achieve,act,adam,add,added,...,yanked,yay,yell,yelled,yelling,yes,yo,youd,youll,youve
ali,1,1,2,0,2,2,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
anthony,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
joe,0,0,0,1,0,0,1,1,0,0,...,1,0,1,0,0,0,1,0,0,3
john,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,0,1,0,1
mike,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
ronny,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,2,1,2,0,0,0


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's apply ldamodel using 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.001*"act" + 0.001*"plays" + 0.001*"paid" + 0.001*"held" + 0.001*"check" + 0.001*"fighting" + 0.001*"cut" + 0.001*"loves" + 0.001*"figure" + 0.001*"opened"'),
 (1,
  '0.010*"walked" + 0.009*"hate" + 0.008*"loved" + 0.006*"understand" + 0.006*"buy" + 0.005*"heard" + 0.005*"happened" + 0.005*"married" + 0.005*"wasnt" + 0.005*"hed"'),
 (2,
  '0.014*"fucking" + 0.011*"fuck" + 0.008*"scared" + 0.008*"eat" + 0.005*"turn" + 0.005*"fun" + 0.005*"hed" + 0.005*"smell" + 0.005*"scammed" + 0.005*"finish"'),
 (3,
  '0.043*"fcking" + 0.015*"fck" + 0.010*"mars" + 0.007*"lived" + 0.006*"figure" + 0.006*"killed" + 0.005*"living" + 0.005*"broke" + 0.005*"worried" + 0.005*"guarding"')]

Topic 0 → Profanity & Online Actions

Topic 1 → Profanity & Strong Expressions

Topic 2 → Daily Life & Actions

Topic 3 → Emotions & Conversations

**Insights from Using Verbs & max_df**

1️ **Shift to Actions & Experiences** – Topics now focus on behaviors, emotions, and interactions rather than just people and objects.

2️ **More Profanity & Expressive Language** – Strong language (e.g., fuck, fcking, hate, killed, scammed) appears in multiple topics, highlighting intense emotions and strong opinions.

3 **Effect of max_df** – Removing very common words helped highlight unique actions and behaviors, making topics more meaningful and distinct.